In [ ]:
!pip install requests
!pip install jsonpath

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# read IMA master list
df = pd.read_csv("tables.csv", header=0, sep=',')
IMA_pdf_data_list = df['Name'].to_list()
len(IMA_pdf_data_list)

In [ ]:
import requests

MINDAT_API_URL = "https://api.mindat.org"
YOUR_API_KEY = ""  

headers = {'Authorization': 'Token '+YOUR_API_KEY}

PAGE_RANGE = 100
for page in range(1, PAGE_RANGE):
  # 'ima': 0/1 all data/IMA data in mindat
  params = {'ima': 1, 'page_size': '20000', 'page': str(page), 'format': 'json'}

  response = requests.get(MINDAT_API_URL+"/items/",params=params, headers=headers)

  json_file = response.json()
  if 1 == page:
    json_all = json_file
  else:
    try:
      json_all['results'] += json_file['results']
    except KeyError:
      break

print("Successfully saved " + str(len(json_all['results'])))
print(len(json_all['results']))

# reference: https://github.com/ChuBL/3DHeatmapDataPreprosses/blob/main/mindat_api.py

In [ ]:
import json
import jsonpath

# read json file
ima_mindat_list = jsonpath.jsonpath(json_all, '$..name')
print(ima_mindat_list)
# print(len(ima_mindat_list))

In [ ]:
# IMA_pdf_data对应ima_mindat_list的差集
IMA_pdf_ima_mindat = set(IMA_pdf_data).difference(set(ima_mindat_list))
print(len(set(IMA_pdf_data).difference(set(ima_mindat_list))))  # 62

# Save in ascending or descending alphabetical order
with open("IMA_pdf_ima_mindat.txt", 'w') as f:
  templist = []
  for t1 in IMA_pdf_ima_mindat:
    templist.append(t1)
  for t2 in sorted(templist, key=str.lower): f.write(str(t2) + '\n')

In [ ]:
tt = set(ima_mindat_list).difference(set(IMA_pdf_data))
print(len(set(ima_mindat_list).difference(set(IMA_pdf_data))))  #  46

# Save in ascending or descending alphabetical order      
with open("ima_mindat_IMA_pdf.txt", 'w') as f:
  templist0 = []
  for t3 in tt:
    templist0.append(t3)
  for t4 in sorted(templist0, key=str.lower): f.write(str(t4) + '\n')

In [ ]:
# Takes the intersection; set is a duplicate-free
set(ima_mindat_list).intersection(set(IMA_pdf_data))

# len(set(ima_mindat_list).intersection(set(IMA_pdf_data)))

In [ ]:
# other method: Get the difference of two lists by looping
a = [x for x in ima_mindat_list if x not in IMA_pdf_data]
b = [y for y in IMA_pdf_data if y not in ima_mindat_list]
print(len(ima_mindat_list))
print(len(IMA_pdf_data))
print(len(a))
print(len(b))